In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

CRIANDO INFORMAÇÕES DOS NOMES DOS ARQUIVOS QUE SERÃO CRIADOS NO NOSSO DIRETÓRIO


In [4]:
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

FUNÇÃO STANDARD DE LEITURA CSV PARA USAR EM OUTRA FUNÇÃO

In [42]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    
    return dataframe

FUNÇÃO STANDARD PARA LEITURA DE ARQUIVO JSON

In [41]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    
    return dataframe

FUNÇÃO STANDARD DE LEITURA XML PARA USAR EM OUTRA FUNÇÃO

In [40]:
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)

    return dataframe

VAMOS EXTRAIR OS VALORES DOS DOCUMENTOS E ARMAZENAR EM UM ARQUIVO ÚNICO

In [43]:
def extract():
    # CRIAMOS UM DATA FRAME VAZIO PARA ACOMODAR OS DADOS DE OUTROS DOCS
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) 
    
    # GLOB PROCURA POR DOCS CSV NO DIRETÓRIO
    for csvfile in glob.glob("dealership_data/*.csv"):
        # USAMOS O DATA FRAME VAZIO E A FUNÇÃO CRIADA ACIMA
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    # GLOB PROCURA POR ARQUIVOS JSON NO DIRETÓRIO
    for jsonfile in glob.glob("dealership_data/*.json"):
        # USAMOS O DATA FRAME VAZIO E A FUNÇÃO CRIADA ACIMA
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    # GLOB PROCURA POR ARQUIVOS XML NO DIRETÓRIO
    for xmlfile in glob.glob("dealership_data/*.xml"):
        # USAMOS O DATA FRAME VAZIO E A FUNÇÃO CRIADA ACIMA
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
     
    return extracted_data
    

FUNÇÃO GENÉRICA PARA TRANSFORMAÇÃO DO DATA FRAME NA COLUNA 'PRICE'

In [47]:
def transform(data):
        data['price'] = round(data.price, 2)
        
        return data

CRIAÇÃO DE UM DOCUMENTO CSV EM NOSSO DIRETÓRIO COM TO_CSV(_)

In [37]:
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)  

FUNÇÃO STANDARD QUE PEGA AS MENSSAGENS QUE CRIAMOS NAS PRIMEIRAS LINHAS PARA INFORMAR LOGS

In [38]:
def log(message):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hour-Minute-Second-MonthName-Day-Year
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n') 

USANDO AS FUNÇÃO CRIADAS ACIMA COM OS VALORES

In [50]:
# INFOMAÇÃO DE DATA E HORA DE COMEÇO DA OPERAÇÃO
log("ETL Job Started")

# INFORMAÇÃO DE DATA E HORA DE INÍCIO E FIM DA EXTRAÇÃO
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")

# INFORMAÇÃO DE INÍCIO E FIM DA EXTRAÇÃO
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

# INFORMAÇÃO DE INÍCIO E FIM DA TRANSFORMAÇÃO
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

# INFORMAÇÃO DE FIM DO PROCESSO ETL
log("ETL Job Ended")


/tmp/ipykernel_12570/1933321689.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_12570/1933321689.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_12570/1933321689.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
/tmp/ipykernel_12570/1933321689.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extracted_data = extracted_data.append(extract_from_

In [49]:
extracted_data



,car_model,year_of_manufacture,price,fuel
0,alto k10,2016,4253.73,Petrol
1,ignis,2017,7313.43,Petrol
2,sx4,2011,6567.16,Petrol
3,alto k10,2014,3731.34,Petrol
4,wagon r,2013,4328.36,Petrol
...,...,...,...,...
85,etios g,2015,5895.52,Petrol
86,corolla altis,2013,8208.96,Petrol
87,corolla,2004,2238.81,Petrol
88,corolla altis,2010,7835.82,Petrol
